In [1]:
import pandas as pd
import pythena
import psycopg2
import numpy as np
from typing import List
import random
import matplotlib.pyplot as plt
import numpy as np
import re
import s3fs

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 1000
    display.max_colwidth = 199
    display.width = 1000
    # display.precision = 2  # set as needed

set_pandas_display_options()
con=pythena.Athena('pdp_dev')

In [2]:
    sql='''

    select sku_uuid as uuid_a, product_segment from pdp_dev.product_warehouse where inserted_date=date('2023-09-18')
    and product_segment='pets' and source_name='chewy' limit 10
    '''
    chewy_df,id=con.execute(sql)

    sql='''
    select sku_uuid as uuid_b from pdp_dev.product_warehouse where 
    inserted_date=date('2023-09-18') and product_segment='pets' and source_name='amazon' limit 10
    '''
    amazon_df,id=con.execute(sql)

    df_1p=pd.DataFrame()
    df_1p['product_segment']='pets'
    df_1p['pair_id']=chewy_df['uuid_a']+'-'+amazon_df['uuid_b']
    df_1p['uuid_a']=chewy_df['uuid_a']
    df_1p['uuid_b']=amazon_df['uuid_b']
    df_1p['score']=1
    df_1p['score']= df_1p['score'].apply(lambda x: random.random())
    df_1p['source_type']='ML'
    df_1p['seller_flag']='1P'

In [3]:
    sql='''

    select sku_uuid as uuid_a, product_segment from pdp_dev.product_warehouse where inserted_date=date('2023-09-18') 
    and product_segment='pets' and source_name='chewy' limit 10
    '''
    chewy_df,id=con.execute(sql)

    sql='''
    select sku_uuid as uuid_b from pdp_dev.product_warehouse where 
    inserted_date=date('2023-09-11') and product_segment='pets' and source_name='amazon' limit 10
    '''
    amazon,id=con.execute(sql)

    df_3p=pd.DataFrame()
    df_3p['product_segment']='pets'
    df_3p['pair_id']=chewy_df['uuid_a']+'-'+amazon['uuid_b']
    df_3p['uuid_a']=chewy_df['uuid_a']
    df_3p['uuid_b']=amazon['uuid_b']
    df_3p['score']=1
    df_3p['score']= df_3p['score'].apply(lambda x: random.random())
    df_3p['source_type']='ML'
    df_3p['seller_flag']='3P'

In [4]:
    sql='''
    select sku_uuid as uuid_a, product_segment from pdp_dev.product_warehouse where inserted_date=date('2023-09-18') 
    and product_segment='pets' and source_name='chewy' limit 10
    '''
    
    
    chewy_df,id=con.execute(sql)

    sql='''
    select sku_uuid as uuid_b from pdp_dev.product_warehouse where 
    inserted_date=date('2023-09-11') and product_segment='pets' and source_name='petco' limit 10
    '''
    petco,id=con.execute(sql)

    new=pd.DataFrame()
    new['pair_id']=chewy_df['uuid_a']+'-'+petco['uuid_b']
    new['uuid_a']=chewy_df['uuid_a']
    new['uuid_b']=petco['uuid_b']
    new['score']=1
    new['score']= df_3p['score'].apply(lambda x: random.random())
    new['source_type']='ML'
    new['seller_flag']='3P'
    new['status']='un_audited'
    new['reason']=''

In [14]:
    delta_df=df_1p.append(df_3p)
    delta_df=delta_df.append(new)
    delta_df['product_segment']='pets'
    delta_df['status']='un_audited'
    delta_df['reason']=''
    delta_df=delta_df.reset_index()
    delta_df.drop('index',axis=1,inplace=True)
    delta_df

C:\Users\geekd\AppData\Local\Temp\ipykernel_27240\4158085891.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  delta_df=df_1p.append(df_3p)
C:\Users\geekd\AppData\Local\Temp\ipykernel_27240\4158085891.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  delta_df=delta_df.append(new)


,product_segment,pair_id,uuid_a,uuid_b,score,source_type,seller_flag,status,reason
0,pets,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,0.403267,ML,1P,un_audited,
1,pets,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,0.827556,ML,1P,un_audited,
2,pets,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,0.238206,ML,1P,un_audited,
3,pets,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,0.953372,ML,1P,un_audited,
4,pets,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,0.482910,ML,1P,un_audited,
5,pets,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,0.830355,ML,1P,un_audited,
6,pets,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.694427,ML,1P,un_audited,
7,pets,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.207040,ML,1P,un_audited,
8,pets,958102<>chewy<>chewy-b0cd5qtld1<>amazon<>amazon,958102<>chewy<>chewy,b0cd5qtld1<>amazon<>amazon,0.107358,ML,1P,un_audited,
9,pets,958702<>chewy<>chewy-b0c9cdpt6d<>amazon<>amazon,958702<>chewy<>chewy,b0c9cdpt6d<>amazon<>amazon,0.443613,ML,1P,un_audited,


In [15]:
ml_type=[]
ml_type.append('ML')


In [44]:
def setTyp(row,typ,score):
    if typ=='ml':
        row['source_type']='ML'
    elif typ=='upc':
        row['source_type']='UPC'
        row['score']=score
    elif typ=='mpn':
        row['source_type']='MPN'
        row['score']=score
    elif typ=='mlupc':
        mlupctp=['ML','UPC']
        row['source_type']=mlupctp
        row['score']=row['score']+1
    elif typ=='mlmpn':
        mlmpntp=['ML','MPN']
        row['source_type']=mlmpntp
        row['score']=row['score']+1
    elif typ=='upcmpn':
        upcmpntp=['UPC','MPN']
        row['source_type']=upcmpntp
        row['score']=2
    elif typ=='all':
        upcmpntp=['UPC','MPN','ML']
        row['source_type']=upcmpntp
        row['score']=row['score']+2
    elif typ=='mlarytype':
        ml_type=[]
        ml_type.append('ML')
        row['source_type']=ml_type
    elif typ=='upcarytype':
        upc_type=[]
        upc_type.append('UPC')
        row['source_type']=upc_type
    elif typ=='mpnarytype':
        mpn_type=[]
        mpn_type.append('MPN')
        row['source_type']=mpn_type
    return row



,product_segment,pair_id,uuid_a,uuid_b,score,source_type,seller_flag
6,pets,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.694427,[ML],1P
7,pets,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.207040,[ML],1P


In [17]:
def setmlTyp(row,typ):
    row['source_type']=typ
    return row
mpn=delta_df[10:15].apply(setTyp,args=('mpn',1),axis=1)
upc=delta_df[15:20].apply(setTyp,args=('upc',1),axis=1)
ml=delta_df[0:10].apply(setTyp,args=('ml',1),axis=1)
ml2=delta_df[20:].apply(setTyp,args=('ml',1),axis=1)


In [18]:
delta_df=ml
delta_df=delta_df.append(mpn)
delta_df=delta_df.append(upc)
delta_df=delta_df.append(ml2)
delta_df

C:\Users\geekd\AppData\Local\Temp\ipykernel_27240\2942399261.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  delta_df=delta_df.append(mpn)
C:\Users\geekd\AppData\Local\Temp\ipykernel_27240\2942399261.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  delta_df=delta_df.append(upc)
C:\Users\geekd\AppData\Local\Temp\ipykernel_27240\2942399261.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  delta_df=delta_df.append(ml2)


,product_segment,pair_id,uuid_a,uuid_b,score,source_type,seller_flag,status,reason
0,pets,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,0.403267,ML,1P,un_audited,
1,pets,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,0.827556,ML,1P,un_audited,
2,pets,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,0.238206,ML,1P,un_audited,
3,pets,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,0.953372,ML,1P,un_audited,
4,pets,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,0.482910,ML,1P,un_audited,
5,pets,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,0.830355,ML,1P,un_audited,
6,pets,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.694427,ML,1P,un_audited,
7,pets,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.207040,ML,1P,un_audited,
8,pets,958102<>chewy<>chewy-b0cd5qtld1<>amazon<>amazon,958102<>chewy<>chewy,b0cd5qtld1<>amazon<>amazon,0.107358,ML,1P,un_audited,
9,pets,958702<>chewy<>chewy-b0c9cdpt6d<>amazon<>amazon,958702<>chewy<>chewy,b0c9cdpt6d<>amazon<>amazon,0.443613,ML,1P,un_audited,


In [28]:
delta_df.to_parquet('s3://mdp-ut.east1/delta/existing.parquet')

In [ ]:
delta_df

# Match

In [20]:
m=delta_df.copy()
m

,product_segment,pair_id,uuid_a,uuid_b,score,source_type,seller_flag,status,reason
0,pets,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,0.403267,ML,1P,un_audited,
1,pets,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,0.827556,ML,1P,un_audited,
2,pets,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,0.238206,ML,1P,un_audited,
3,pets,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,0.953372,ML,1P,un_audited,
4,pets,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,0.482910,ML,1P,un_audited,
5,pets,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,0.830355,ML,1P,un_audited,
6,pets,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.694427,ML,1P,un_audited,
7,pets,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.207040,ML,1P,un_audited,
8,pets,958102<>chewy<>chewy-b0cd5qtld1<>amazon<>amazon,958102<>chewy<>chewy,b0cd5qtld1<>amazon<>amazon,0.107358,ML,1P,un_audited,
9,pets,958702<>chewy<>chewy-b0c9cdpt6d<>amazon<>amazon,958702<>chewy<>chewy,b0c9cdpt6d<>amazon<>amazon,0.443613,ML,1P,un_audited,


In [21]:
m=m.drop(['status','reason'],axis=1)

In [22]:
m

,product_segment,pair_id,uuid_a,uuid_b,score,source_type,seller_flag
0,pets,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,0.403267,ML,1P
1,pets,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,0.827556,ML,1P
2,pets,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,0.238206,ML,1P
3,pets,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,0.953372,ML,1P
4,pets,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,0.482910,ML,1P
5,pets,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,0.830355,ML,1P
6,pets,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.694427,ML,1P
7,pets,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.207040,ML,1P
8,pets,958102<>chewy<>chewy-b0cd5qtld1<>amazon<>amazon,958102<>chewy<>chewy,b0cd5qtld1<>amazon<>amazon,0.107358,ML,1P
9,pets,958702<>chewy<>chewy-b0c9cdpt6d<>amazon<>amazon,958702<>chewy<>chewy,b0c9cdpt6d<>amazon<>amazon,0.443613,ML,1P


In [45]:
mlupc=m[0:2].apply(setTyp,args=('mlupc',1),axis=1)
print(mlupc)
mlmpn=m[2:4].apply(setTyp,args=('mlmpn',1),axis=1)
print(mlmpn)
mum=m[4:6].apply(setTyp,args=('all',1),axis=1)
print(mum)
um=m[14:16].apply(setTyp,args=('upcmpn',1),axis=1)
print(um)

mlm=m[6:8].apply(setTyp,args=('ml',1),axis=1)
mlm=m[6:8].apply(setTyp,args=('mlarytype',1),axis=1)
u=m[17:19].apply(setTyp,args=('upcarytype',1),axis=1)
mpn=m[11:13].apply(setTyp,args=('mpnarytype',1),axis=1)

  product_segment                                          pair_id                uuid_a                      uuid_b     score source_type seller_flag
0            pets  963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon  963846<>chewy<>chewy  b0bzyyqbj2<>amazon<>amazon  1.403267   [ML, UPC]          1P
1            pets  958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon  958478<>chewy<>chewy  b0c8z2vx6h<>amazon<>amazon  1.827556   [ML, UPC]          1P
  product_segment                                          pair_id                uuid_a                      uuid_b     score source_type seller_flag
2            pets  956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon  956854<>chewy<>chewy  b0ch1ls52w<>amazon<>amazon  1.238206   [ML, MPN]          1P
3            pets  963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon  963702<>chewy<>chewy  b0bh32zck8<>amazon<>amazon  1.953372   [ML, MPN]          1P
  product_segment                                          pair_id                uuid_a      

,product_segment,pair_id,uuid_a,uuid_b,score,source_type,seller_flag
6,pets,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.694427,ML,1P
7,pets,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.207040,ML,1P


In [46]:
finalmatch=mlupc
finalmatch=finalmatch.append(mlmpn).append(mum).append(um).append(mlm).append(u).append(mpn)
finalmatch

C:\Users\geekd\AppData\Local\Temp\ipykernel_27240\670742159.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalmatch=finalmatch.append(mlmpn).append(mum).append(um).append(mlm).append(u).append(mpn)
C:\Users\geekd\AppData\Local\Temp\ipykernel_27240\670742159.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalmatch=finalmatch.append(mlmpn).append(mum).append(um).append(mlm).append(u).append(mpn)
C:\Users\geekd\AppData\Local\Temp\ipykernel_27240\670742159.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finalmatch=finalmatch.append(mlmpn).append(mum).append(um).append(mlm).append(u).append(mpn)


,product_segment,pair_id,uuid_a,uuid_b,score,source_type,seller_flag
0,pets,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,1.403267,"[ML, UPC]",1P
1,pets,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,1.827556,"[ML, UPC]",1P
2,pets,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,1.238206,"[ML, MPN]",1P
3,pets,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,1.953372,"[ML, MPN]",1P
4,pets,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,2.482910,"[UPC, MPN, ML]",1P
5,pets,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,2.830355,"[UPC, MPN, ML]",1P
14,pets,956998<>chewy<>chewy-b08nc5c2wl<>amazon<>amazon,956998<>chewy<>chewy,b08nc5c2wl<>amazon<>amazon,2.000000,"[UPC, MPN]",3P
15,pets,957190<>chewy<>chewy-b09mq5v2v8<>amazon<>amazon,957190<>chewy<>chewy,b09mq5v2v8<>amazon<>amazon,2.000000,"[UPC, MPN]",3P
6,pets,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.694427,[ML],1P
7,pets,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.207040,[ML],1P


In [53]:
def settyp(row):
    l=[]
    for r in row['source_type']:
        tl=[]
        tl.append(r)
        if(r=='ML'):
            tl.append(str(row.score-int(row.score)))
        else:
            tl.append(str(1))
        print(tl)
        l.append(tl)
        row['typ_score']=l
    return row
finalmatch=finalmatch.apply(settyp,axis=1)
finalmatch.rename(columns={'score': 'aggregated_score', 'source_type': 'source_type_list','typ_score': 'type_score_list'}, inplace=True)
finalmatch

KeyError: 'source_type'

In [55]:
finalmatch

,product_segment,pair_id,uuid_a,uuid_b,aggregated_score,source_type_list,seller_flag,type_score_list
0,pets,963846<>chewy<>chewy-b0bzyyqbj2<>amazon<>amazon,963846<>chewy<>chewy,b0bzyyqbj2<>amazon<>amazon,1.403267,"[ML, UPC]",1P,"[[ML, 0.40326678730236587], [UPC, 1]]"
1,pets,958478<>chewy<>chewy-b0c8z2vx6h<>amazon<>amazon,958478<>chewy<>chewy,b0c8z2vx6h<>amazon<>amazon,1.827556,"[ML, UPC]",1P,"[[ML, 0.8275559477460037], [UPC, 1]]"
2,pets,956854<>chewy<>chewy-b0ch1ls52w<>amazon<>amazon,956854<>chewy<>chewy,b0ch1ls52w<>amazon<>amazon,1.238206,"[ML, MPN]",1P,"[[ML, 0.2382059557836267], [MPN, 1]]"
3,pets,963702<>chewy<>chewy-b0bh32zck8<>amazon<>amazon,963702<>chewy<>chewy,b0bh32zck8<>amazon<>amazon,1.953372,"[ML, MPN]",1P,"[[ML, 0.953371890300913], [MPN, 1]]"
4,pets,956998<>chewy<>chewy-b0cbh8x5v3<>amazon<>amazon,956998<>chewy<>chewy,b0cbh8x5v3<>amazon<>amazon,2.482910,"[UPC, MPN, ML]",1P,"[[UPC, 1], [MPN, 1], [ML, 0.4829100357697742]]"
5,pets,957190<>chewy<>chewy-b0bzkpr9sq<>amazon<>amazon,957190<>chewy<>chewy,b0bzkpr9sq<>amazon<>amazon,2.830355,"[UPC, MPN, ML]",1P,"[[UPC, 1], [MPN, 1], [ML, 0.8303549250527285]]"
14,pets,956998<>chewy<>chewy-b08nc5c2wl<>amazon<>amazon,956998<>chewy<>chewy,b08nc5c2wl<>amazon<>amazon,2.000000,"[UPC, MPN]",3P,"[[UPC, 1], [MPN, 1]]"
15,pets,957190<>chewy<>chewy-b09mq5v2v8<>amazon<>amazon,957190<>chewy<>chewy,b09mq5v2v8<>amazon<>amazon,2.000000,"[UPC, MPN]",3P,"[[UPC, 1], [MPN, 1]]"
6,pets,959870<>chewy<>chewy-b0c1bvbqp2<>amazon<>amazon,959870<>chewy<>chewy,b0c1bvbqp2<>amazon<>amazon,0.694427,[ML],1P,"[[ML, 0.6944265995757058]]"
7,pets,959782<>chewy<>chewy-b0chbc3kz1<>amazon<>amazon,959782<>chewy<>chewy,b0chbc3kz1<>amazon<>amazon,0.207040,[ML],1P,"[[ML, 0.20703960200323546]]"


In [56]:
finalmatch.to_parquet('s3://mdp-ut.east1/match/existing.parquet')

# oct-10